# Hillslope Scale Calibration Tutorial

This notebook outlines how to perform model calibration for a selected subset of REWs. It is assumed that the selected subset is a unique sub-watershed of the full model watershed. It is furthermore assumed that channel effects are not important at the scale of the sub-watershed, so that the model can be compared to streamflow data simply by up-scaling the hillslopes output. 

Two files are required for hillslope scale calibration:

1. A shapefile corresponding to the sub-basin to be calibrated must be stored in `raw_data/watershed_poly`. 
2. Streamflow data (in units of cm/day) stored in the `calibration_data` folder. This must be gapless, daily streamflow data spanning at least the time period from `spinup_date` to `stop_date`. 

Full model calibration including channel transport is overviewed in the [Network Scale Calibration Tutorial](./network_scale_calibration.ipynb). 

In [ ]:
%matplotlib inline
import os
import sys
from os.path import dirname
parent_dir = dirname(dirname(os.getcwd()))
sys.path.append(os.path.join(parent_dir,'StreamflowTempModel','2_hillslope_discharge'))
sys.path.append(os.path.join(parent_dir,'StreamflowTempModel','3_channel_routing'))

from vadoseZone import *
from groundwaterZone import *
from REW import REW
from matplotlib import pyplot as plt
import numpy as np
import seaborn as sns
import pickle
from datetime import date
import pandas as pd
import numpy as np
import time
import sys

# Load config files, forcing file, and paramters for each group
parent_dir = os.path.dirname(os.path.dirname(os.getcwd()))

sys.path.append(os.path.join(parent_dir, 'StreamflowTempModel', '1_data_preparation'))
from prep import rew_params
rew_params()

# These dictionaries contain the all the data we'll need to instantiate 
rew_config = pickle.load( open( os.path.join(parent_dir,'model_data','rew_config.p'), "rb" ) )
rew_forcing = pickle.load( open( os.path.join(parent_dir,'model_data','rew_forcing.p'), "rb" ) )
group_params = pickle.load( open( os.path.join(parent_dir,'model_data','group_params.p'), "rb" ))
model_config = pickle.load( open( os.path.join(parent_dir, 'model_data', 'model_config.p'), 'rb'))
param_ranges = pickle.load( open( os.path.join(parent_dir, 'model_data', 'model_config.p'), 'rb'))

start_date = model_config['start_date']
stop_date = model_config['stop_date']
spinup_date = model_config['spinup_date']
Tmax = model_config['Tmax']
dt = model_config['dt_hillslope']
t = model_config['t_hillslope']
resample_freq_hillslope = model_config['resample_freq_hillslope']
timestamps_hillslope = model_config['timestamps_hillslope']

In [ ]:
solved_group_hillslopes_dict = {}
for group_id in group_params.keys():
    ## find the first REW that belongs to the desired group
    rew_id = next(rew_id for rew_id in rew_config.keys() if rew_config[rew_id]['group'] == group_id)
    
    vz = group_params[group_id]['vz'](**group_params[group_id])
    gz = group_params[group_id]['gz'](**group_params[group_id])    
    
    rew = REW(vz, gz,  **{'pet':rew_forcing[rew_id].pet, 'ppt':rew_forcing[rew_id].ppt, 'aspect':90})

    storage    = np.zeros(np.size(t))
    groundwater     = np.zeros(np.size(t))
    discharge       = np.zeros(np.size(t))
    leakage         = np.zeros(np.size(t))
    ET              = np.zeros(np.size(t))

    # Resample pet and ppt to integration timestep
    ppt = np.array(rew.ppt[start_date:stop_date].resample(resample_freq_hillslope).ffill())
    pet = np.array(rew.pet[start_date:stop_date].resample(resample_freq_hillslope).ffill())

    # Solve REW hillslope
    for i in range(len(t)):
        rew.vz.update(dt,**{'ppt':ppt[i],'pet':pet[i]})
        storage[i] = rew.vz.storage
        leakage[i]      = rew.vz.leakage
        ET[i]           = rew.vz.ET   
        rew.gz.update(dt,**{'leakage':leakage[i]})
        groundwater[i] = rew.gz.storage
        discharge[i] = rew.gz.discharge

    # Save all results as daily data. 
    solved_hillslope = pd.DataFrame({'storage':storage, 'leakage':leakage, 'ET':ET, 'groundwater':groundwater, 'discharge':discharge}, index=timestamps_hillslope)
    solved_group_hillslopes_dict[group_id] = solved_hillslope.resample('D').mean()

pickle.dump( solved_group_hillslopes_dict, open( os.path.join(parent_dir,'model_data','solved_hillslope_discharge.p'), "wb" ) )